In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.linear_model import ElasticNet

In [2]:
concrete_df=pd.read_csv('Concrete_Data.csv')

## ElasticNet

In [3]:
concrete_df

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
X=concrete_df.drop('Strength',axis=1)

In [5]:
y=concrete_df['Strength']

In [6]:
kfold=KFold(n_splits=5,shuffle=True,random_state=23)

In [7]:
elastic=ElasticNet()

In [8]:
elastic.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [9]:
params={'alpha':np.linspace(0.001,5,5),'l1_ratio':np.linspace(0,1,5)}

In [10]:
gcv=GridSearchCV(elastic,param_grid=params,cv=kfold,scoring='r2')

In [11]:
gcv.fit(X,y)

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.522e+04, tolerance: 2.258e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.313e+04, tolerance: 2.326e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in s

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.555e+04, tolerance: 2.872e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=ElasticNet(),
             param_grid={'alpha': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ])},
             scoring='r2')

In [12]:
gcv.best_params_

{'alpha': 5.0, 'l1_ratio': 0.0}

In [13]:
elastic_best_score=gcv.best_score_
elastic_best_score

0.6006431793947039

## 2. Scaling with k-NN

In [14]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [15]:
scaler=StandardScaler()

In [16]:
knn=KNeighborsRegressor()

In [17]:
pipe=Pipeline([('SCL',scaler),('KNN',knn)])

In [18]:
pipe.get_params()

{'memory': None,
 'steps': [('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())],
 'verbose': False,
 'SCL': StandardScaler(),
 'KNN': KNeighborsRegressor(),
 'SCL__copy': True,
 'SCL__with_mean': True,
 'SCL__with_std': True,
 'KNN__algorithm': 'auto',
 'KNN__leaf_size': 30,
 'KNN__metric': 'minkowski',
 'KNN__metric_params': None,
 'KNN__n_jobs': None,
 'KNN__n_neighbors': 5,
 'KNN__p': 2,
 'KNN__weights': 'uniform'}

In [19]:
params={'SCL':[StandardScaler(),MinMaxScaler()],'KNN__n_neighbors':[1,2,3,4]}

In [20]:
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='r2')

In [21]:
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('KNN', KNeighborsRegressor())]),
             param_grid={'KNN__n_neighbors': [1, 2, 3, 4],
                         'SCL': [StandardScaler(), MinMaxScaler()]},
             scoring='r2')

In [22]:
gcv.best_params_

{'KNN__n_neighbors': 4, 'SCL': StandardScaler()}

In [23]:
KNN_best_score=gcv.best_score_
KNN_best_score

0.715311710031387

## 3. DTR

In [24]:
from sklearn.tree import DecisionTreeRegressor

In [25]:
dtr=DecisionTreeRegressor()

In [26]:
dtr.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [27]:
params={'max_depth':[None,3,5],'min_samples_split':[2,5,10],'min_samples_leaf':[1,5,10]}

In [28]:
gcv=GridSearchCV(dtr,param_grid=params,cv=kfold,scoring='r2')

In [29]:
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [None, 3, 5],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 5, 10]},
             scoring='r2')

In [30]:
gcv.best_params_

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [31]:
DTR_best_score=gcv.best_score_
DTR_best_score

0.8479229409995318

## Model_Ensambling

In [32]:
from sklearn.ensemble import VotingRegressor

In [43]:
ensamble=VotingRegressor(estimators=[('Elastic',elastic),('KNN',pipe),('DTR',dtr)],verbose=3,weights=[0.6006431793947039,0.715311710031387,0.8479229409995318])

In [44]:
ensamble.get_params()

{'estimators': [('Elastic', ElasticNet()),
  ('KNN',
   Pipeline(steps=[('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())])),
  ('DTR', DecisionTreeRegressor())],
 'n_jobs': None,
 'verbose': 3,
 'weights': [0.6006431793947039, 0.715311710031387, 0.8479229409995318],
 'Elastic': ElasticNet(),
 'KNN': Pipeline(steps=[('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())]),
 'DTR': DecisionTreeRegressor(),
 'Elastic__alpha': 1.0,
 'Elastic__copy_X': True,
 'Elastic__fit_intercept': True,
 'Elastic__l1_ratio': 0.5,
 'Elastic__max_iter': 1000,
 'Elastic__positive': False,
 'Elastic__precompute': False,
 'Elastic__random_state': None,
 'Elastic__selection': 'cyclic',
 'Elastic__tol': 0.0001,
 'Elastic__warm_start': False,
 'KNN__memory': None,
 'KNN__steps': [('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())],
 'KNN__verbose': False,
 'KNN__SCL': StandardScaler(),
 'KNN__KNN': KNeighborsRegressor(),
 'KNN__SCL__copy': True,
 'KNN__SCL__with_mean': True,
 'KNN__SCL__with_

In [45]:
params={'Elastic__alpha':np.linspace(0.001,5,5),'Elastic__l1_ratio':np.linspace(0,1,5),'KNN__SCL':[StandardScaler(),MinMaxScaler()],'KNN__KNN__n_neighbors':[1,2,3,4],'DTR__max_depth':[None,3,5],'DTR__min_samples_split':[2,5,10],'DTR__min_samples_leaf':[1,5,10]}

In [46]:
from sklearn.model_selection import RandomizedSearchCV

In [47]:
rgcv=RandomizedSearchCV(ensamble,param_distributions=params,n_iter=100,cv=kfold,verbose=3,random_state=23)

In [48]:
rgcv.fit(X,y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 1/5] END DTR__max_depth=5, DTR__min_samples_leaf=1, DTR__min_samples_split=10, Elastic__alpha=2.5004999999999997, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=1, KNN__SCL=StandardScaler();, score=0.843 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 2/5] END DTR__max_depth=5, DTR__min_samples_leaf=1, DTR__min_samples_split=10, Elastic__alpha=2.5004999999999997, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=1, KNN__SCL=StandardScaler();, score=0.833 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, to

[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 1/5] END DTR__max_depth=None, DTR__min_samples_leaf=1, DTR__min_samples_split=5, Elastic__alpha=1.2507499999999998, Elastic__l1_ratio=0.5, KNN__KNN__n_neighbors=3, KNN__SCL=StandardScaler();, score=0.854 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 2/5] END DTR__max_depth=None, DTR__min_samples_leaf=1, DTR__min_samples_split=5, Elastic__alpha=1.2507499999999998, Elastic__l1_ratio=0.5, KNN__KNN__n_neighbors=3, KNN__SCL=StandardScaler();, score=0.843 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processi

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.541e+04, tolerance: 2.258e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.338e+04, tolerance: 2.326e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in s

[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 1/5] END DTR__max_depth=None, DTR__min_samples_leaf=1, DTR__min_samples_split=10, Elastic__alpha=0.001, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=4, KNN__SCL=MinMaxScaler();, score=0.841 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 2/5] END DTR__max_depth=None, DTR__min_samples_leaf=1, DTR__min_samples_split=10, Elastic__alpha=0.001, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=4, KNN__SCL=MinMaxScaler();, score=0.834 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 

[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 5/5] END DTR__max_depth=5, DTR__min_samples_leaf=10, DTR__min_samples_split=10, Elastic__alpha=3.75025, Elastic__l1_ratio=0.25, KNN__KNN__n_neighbors=1, KNN__SCL=StandardScaler();, score=0.780 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 1/5] END DTR__max_depth=None, DTR__min_samples_leaf=5, DTR__min_samples_split=10, Elastic__alpha=3.75025, Elastic__l1_ratio=0.5, KNN__KNN__n_neighbors=1, KNN__SCL=MinMaxScaler();, score=0.854 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 2/5] END DTR__max_depth=None, DTR__min_samples_leaf=5, DTR__min_samp

...

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.547e+04, tolerance: 2.258e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.346e+04, tolerance: 2.326e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in s

[CV 3/5] END DTR__max_depth=3, DTR__min_samples_leaf=1, DTR__min_samples_split=2, Elastic__alpha=5.0, Elastic__l1_ratio=0.0, KNN__KNN__n_neighbors=2, KNN__SCL=MinMaxScaler();, score=0.806 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 4/5] END DTR__max_depth=3, DTR__min_samples_leaf=1, DTR__min_samples_split=2, Elastic__alpha=5.0, Elastic__l1_ratio=0.0, KNN__KNN__n_neighbors=2, KNN__SCL=MinMaxScaler();, score=0.791 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 5/5] END DTR__max_depth=3, DTR__min_samples_leaf=1, DTR__min_samples_split=2, Elastic__alpha=5.0, Elastic__l1_ratio=0.0, KNN__KNN__n_neighbors=2, KNN__SCL=M

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.528e+04, tolerance: 2.258e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.322e+04, tolerance: 2.326e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in s

[CV 2/5] END DTR__max_depth=3, DTR__min_samples_leaf=10, DTR__min_samples_split=10, Elastic__alpha=0.001, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=4, KNN__SCL=MinMaxScaler();, score=0.746 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 3/5] END DTR__max_depth=3, DTR__min_samples_leaf=10, DTR__min_samples_split=10, Elastic__alpha=0.001, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbors=4, KNN__SCL=MinMaxScaler();, score=0.792 total time=   0.0s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.0s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.0s
[CV 4/5] END DTR__max_depth=3, DTR__min_samples_leaf=10, DTR__min_samples_split=10, Elastic__alpha=0.001, Elastic__l1_ratio=0.75, KNN__KNN__n_neighbor

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
                   estimator=VotingRegressor(estimators=[('Elastic',
                                                          ElasticNet()),
                                                         ('KNN',
                                                          Pipeline(steps=[('SCL',
                                                                           StandardScaler()),
                                                                          ('KNN',
                                                                           KNeighborsRegressor())])),
                                                         ('DTR',
                                                          DecisionTreeRegressor())],
                                             verbose=3,
                                             weights=[0.6006431793947039,
                                                      0.715311710031387,
                                                      0.8479229409995318]),
                   n_iter=100,
                   param_distributions={'DTR__max_depth': [None, 3, 5],
                                        'DTR__min_samples_leaf': [1, 5, 10],
                                        'DTR__min_samples_split': [2, 5, 10],
                                        'Elastic__alpha': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                                        'Elastic__l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
                                        'KNN__KNN__n_neighbors': [1, 2, 3, 4],
                                        'KNN__SCL': [StandardScaler(),
                                                     MinMaxScaler()]},
                   random_state=23, verbose=3)

In [49]:
rgcv.best_params_

{'KNN__SCL': StandardScaler(),
 'KNN__KNN__n_neighbors': 1,
 'Elastic__l1_ratio': 1.0,
 'Elastic__alpha': 1.2507499999999998,
 'DTR__min_samples_split': 2,
 'DTR__min_samples_leaf': 1,
 'DTR__max_depth': None}

In [50]:
rgcv.best_score_

0.8600723209625863

In [51]:
unweighted_score=gcv.best_score_

In [54]:
weighted_score=rgcv.best_score_
weighted_score

0.8600723209625863

In [52]:
print('unweighted_best_score : ',unweighted_score)
print('elastic_best_score : ',elastic_best_score)
print('KNN_best_score : ',KNN_best_score)
print('DTR_best_score : ',DTR_best_score)

unweighted_best_score :  0.8479229409995318
elastic_best_score :  0.6006431793947039
KNN_best_score :  0.715311710031387
DTR_best_score :  0.8479229409995318


In [55]:
print('weighted_best_score : ',weighted_score)

weighted_best_score :  0.8600723209625863
